<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

### **Extracción de datos directamente de archivos con Spark SQL**

En este notebook, aprenderás a extraer datos directamente de archivos usando Spark SQL en Databricks.

Varios formatos de archivo soportan esta opción, pero es más útil para formatos de datos autodescriptivos (como Parquet y JSON).

#### **Objetivos de aprendizaje**
Al final de esta lección, deberías ser capaz de:
- Utilizar Spark SQL para consultar directamente archivos de datos
- Layer views y CTEs para facilitar la consulta de archivos de datos
- Aprovechar los métodos **`text`** y **`binaryFile`** para revisar el contenido de archivos sin procesar

#### **Run Setup**

El setup script creará los datos y declarará los valores necesarios para que el resto de este notebook se ejecute.

In [ ]:
%run ../Includes/Classroom-Setup-02.1

#### **Data Overview**

En este ejemplo, trabajaremos con una muestra de datos Kafka sin procesar (raw Kafka data) escritos como archivos JSON. 

Cada archivo contiene todos los registros consumidos durante un intervalo de 5 segundos, almacenados con el schema completo de Kafka como un archivo JSON de múltiples registros (multiple-record JSON file).

| field | type | description |
| --- | --- | --- |
| key | BINARY | The **`user_id`** field is used as the key; this is a unique alphanumeric field that corresponds to session/cookie information |
| value | BINARY | This is the full data payload (to be discussed later), sent as JSON |
| topic | STRING | While the Kafka service hosts multiple topics, only those records from the **`clickstream`** topic are included here |
| partition | INTEGER | Our current Kafka implementation uses only 2 partitions (0 and 1) |
| offset | LONG | This is a unique value, monotonically increasing for each partition |
| timestamp | LONG | This timestamp is recorded as milliseconds since epoch, and represents the time at which the producer appends a record to a partition |

Note that our source directory contains many JSON files.

In [ ]:
%python
print(DA.paths.kafka_events)

files = dbutils.fs.ls(DA.paths.kafka_events)
display(files)

<center><img src="https://i.postimg.cc/15ScMR8L/db337.png"></center>

En este caso, utilizaremos rutas de archivo relativas a los datos que se han escrito en el DBFS root. 

La mayoría de workflows requerirán que los usuarios accedan a datos desde ubicaciones externas de almacenamiento en la nube. 

En la mayoría de las empresas, un administrador del workspace será responsable de configurar el acceso a estas ubicaciones de almacenamiento.

Las instrucciones para configurar y acceder a estas ubicaciones se pueden encontrar en los cursos autodidácticos específicos para proveedores de nube titulados " "Cloud Architecture & Systems Integrations" (Arquitectura de nube e integraciones de sistemas).

#### **Consultar un único archivo**

Para consultar los datos contenidos en un único archivo, ejecute la consulta con el siguiente patrón:

<strong><code>SELECT * FROM file_format.&#x60;/path/to/file&#x60;</code></strong>

Preste especial atención al uso de comillas (no simples) alrededor de la ruta.

In [ ]:
%sql
SELECT * FROM json.`${DA.paths.kafka_events}/001.json`

Observe que nuestra vista previa muestra las 321 filas de nuestro archivo fuente.

<center><img src="https://i.postimg.cc/k4dB9GPC/db338.png"></center>

#### **Consultar un directorio de archivos**

Suponiendo que todos los archivos de un directorio tengan el mismo formato y schema, todos los archivos pueden consultarse simultáneamente especificando la ruta del directorio en lugar de un archivo individual.

In [ ]:
%sql
SELECT * FROM json.`${DA.paths.kafka_events}`

Por defecto, esta consulta sólo mostrará las 1000 primeras filas.

<center><img src="https://i.postimg.cc/gk0rG1Qx/db339.png"></center>

#### **Crear referencias a archivos**

Esta capacidad de consultar directamente archivos y directorios significa que se puede encadenar lógica Spark adicional a consultas contra archivos.

Cuando creamos una vista a partir de una consulta sobre una ruta, podemos hacer referencia a esta vista en consultas posteriores.

In [ ]:
%sql
CREATE OR REPLACE VIEW event_view
AS 
SELECT * FROM json.`${DA.paths.kafka_events}`

Siempre que un usuario tenga permiso para acceder a la vista y a la ubicación de almacenamiento subyacente, podrá utilizar esta definición de vista para consultar los datos subyacentes. Esto se aplica a diferentes usuarios del workspace, diferentes notebooks y diferentes clusters.

In [ ]:
%sql
SELECT * FROM event_view

<center><img src="https://i.postimg.cc/vHSTT3w3/db340.png"></center>

#### **Crear referencias temporales a archivos**

Las vistas temporales permiten asignar a las consultas un nombre que sea más fácil de referenciar en consultas posteriores.

In [ ]:
%sql
CREATE OR REPLACE TEMP VIEW events_temp_view
AS 
SELECT * FROM json.`${DA.paths.kafka_events}`

Las vistas temporales sólo existen para la SparkSession actual. En Databricks, esto significa que están aisladas del notebook, job o consulta DBSQL actual.

In [ ]:
%sql
SELECT * FROM events_temp_view

<center><img src="https://i.postimg.cc/8zJ56M1D/db341.png"></center>

#### **Aplicar CTEs como referencia dentro de una consulta**

Las Common table expressions (CTE) son perfectas cuando se desea una referencia de corta duración y legible por el usuario a los resultados de una consulta.

In [ ]:
%sql
WITH cte_json
AS (SELECT * FROM json.`${DA.paths.kafka_events}`)
SELECT * FROM cte_json

<center><img src="https://i.postimg.cc/wjCBcMvN/db342.png"></center>

Los CTEs sólo aliasan (de 'alias') los resultados de una consulta mientras esa consulta está siendo planificada y ejecutada.

Por lo tanto, **la siguiente celda arrojará un error cuando se ejecute**.

In [ ]:
%sql
-- SELECT COUNT(*) FROM cte_json

<center><img src="https://i.postimg.cc/v8t6YrmN/db343.png"></center>

#### **Extraer archivos de texto como strings sin procesar (raw strings)**

Al trabajar con archivos basados en texto (que incluyen los formatos JSON, CSV, TSV y TXT), puede utilizar el formato **`text`** para cargar cada línea del archivo como una fila con una columna de cadena denominada **`value`**. Esto puede ser útil cuando las fuentes de datos son propensas a la corrupción y se utilizarán funciones personalizadas de análisis de texto para extraer valores de los campos de texto.

In [ ]:
%sql
SELECT * FROM text.`${DA.paths.kafka_events}`

<center><img src="https://i.postimg.cc/85DP9YLP/db344.png"></center>

#### **Extraer los raw bytes y los metadatos de un archivo**

Algunos workflows pueden requerir trabajar con archivos enteros, como cuando se trabaja con imágenes o datos no estructurados. El uso de **`binaryFile`** para consultar un directorio proporcionará metadatos del archivo junto con la representación binaria del contenido del archivo.

En concreto, los campos creados indicarán el **`path`**, **`modificationTime`**, **`length`**, y **`content`**.

In [ ]:
%sql
SELECT * FROM binaryFile.`${DA.paths.kafka_events}`

<center><img src="https://i.postimg.cc/J0r0DFhd/db345.png"></center>

Ejecute la siguiente celda para eliminar las tablas y archivos asociados a esta lección.

In [ ]:
%python 
DA.cleanup()

<center><img src="https://i.postimg.cc/MZ5v2zPk/db346.png"></center>

-sandbox
&copy; 2022 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="https://help.databricks.com/">Support</a>